In [17]:
import pandas as pd
import numpy as np
import os
import kagglehub
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [18]:
import re

TITLE_PATTERN = re.compile(r'\D(19[0-9]{2}|20[0-9]{2})')

def vintage(title):
  m = TITLE_PATTERN.findall(title) 
  if len(m) > 0:
    m.sort()
    return int(m[-1])
  return None

RATINGS = [('acceptable', range(80,83)),('good', range(83,87)),('very good', range(87,90)),('excellent', range(90,94)),('superb', range(94,98)),('classic', range(98,101))]

In [19]:
country = 'US'
province = 'California'
variety = 'Pinot Noir'
cols = ['title', 'description', 'winery', 'points']

path = kagglehub.dataset_download('christopheiv/winemagdata130k')
fname = 'winemag-data-130k-v2.csv'
reviews = pd.read_csv(os.path.join(path, fname), index_col=0).query(f'country == "{country}" and province == "{province}" and variety == "{variety}"')[cols]
reviews['year'] = reviews.title.apply(vintage)
reviews.drop(['title'], axis=1, inplace=True)
reviews['rating'] = reviews.points.map({points: rating for rating, points_range in RATINGS for points in points_range})
reviews.dropna(subset=['year'], axis=0, inplace=True)
reviews.year = reviews.year.astype(int)
reviews['review_len'] = reviews.description.str.len()
reviews.info()
reviews.head()

<class 'pandas.core.frame.DataFrame'>
Index: 6888 entries, 25 to 129920
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  6888 non-null   object
 1   winery       6888 non-null   object
 2   points       6888 non-null   int64 
 3   year         6888 non-null   int64 
 4   rating       6888 non-null   object
 5   review_len   6888 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 376.7+ KB


,description,winery,points,year,rating,review_len
25,Oak and earth intermingle around robust aromas...,Castello di Amorosa,87,2011,very good,259
145,"Cooked cranberry is spiced with anise, pepperc...",Testarossa,91,2013,excellent,267
152,"From a promising new project, the nose of this...",Albatross Ridge,91,2012,excellent,351
169,This wine is put together from multiple vineya...,Dutton-Goldfield,91,2014,excellent,327
181,"Crisp, coastal acidity dominates this wine, ma...",McIntyre Vineyards,88,2006,very good,219


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2', trust_remote_code=False)
embeddings = model.encode(reviews.description.to_list(), normalize_embeddings=True, show_progress_bar=True, device='cpu')

Batches:   0%|          | 0/216 [00:00<?, ?it/s]

In [ ]:
embeddings_df = pd.DataFrame(embeddings, index=reviews.index)
embeddings_df.columns = embeddings_df.columns.astype(str)
embeddings_df.info()
embeddings_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding
from tensorflow.keras.layers import Dropout, Activation, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.utils import to_categorical

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

